## Introduction to parallel computing in Python

The designers of the Python language made the choice that only one thread in a process can run actual Python code by using the so-called global interpreter lock (GIL). This means that approaches that may work in other languages (C, C++, Fortran), may not work in Python. At first glance, this is bad for parallelism. But it’s not all bad!:

    External libraries (NumPy, SciPy, Pandas, etc), written in C or other languages, can release the lock and run multi-threaded.

    Most input/output releases the GIL, and input/output is slow. The threading library can be used to multithread I/O.

    Python libraries like multiprocessing and mpi4py run multiple Python processes and this circumvents the GIL.


## Shared Memory
The main library that we will be using is `multiprocessing`. This library has plenty of utilities to develop multiple parallelization architectures in Python. We start importing it.

In [ ]:
import multiprocessing as mp
import os

This library has a similar usage to `map` native Python's method.

In [ ]:
def square(x):
    s = f"PID = {os.getpid()}"
    print(s)
    return x*x

In [ ]:
list(map(square, [2,3,14,56,7]))

The pool class, by default, creates one new process per CPU and does parallel calculations on the list:

In [ ]:
with mp.Pool(processes=6) as pool:
    a = pool.map(square, [1, 2, 3, 4, 5, 6])

 Pool by default uses one process for each CPU on the node - it doesn’t know about your cluster’s scheduling system. It’s possible that you have permission to use 2 CPUs but it is trying to use 12. This is generally a bad situation, and will just slow you down (and make other users on the same node upset)!

## Example (Prime Numbers)
Let's say that we want to count how many primes are between $0$ and $n$, where $n\in\mathbb{N}$. For that, we first define a function that checks an array of numbers and returns the count of how many of them are prime.

In [ ]:
def count_primes(shared_integer:mp.Value, numbers_list:list):
    """
    This function counts how many integers are in numbers_list, and add them
    to the shared variable shared_integer.
    """
    for number in numbers_list:
        isPrime = True # We assume that is prime
        for factor in range(2, int(number**0.5)+1):
            if number % factor == 0:
                isPrime = False
                break
        if isPrime:
            shared_integer.value += 1

And now we write the parallelization part.

In [ ]:
def parallel_count_primes(n:int, m:int):
    """
    This function uses parallel programming (shared memory architecture)
    to count how many primes are between 0 and n, using m processors.
    """
    # We define first the shared integer
    shared_integer = mp.Value("i", 0) # (i)nteger, starting at 0
    
    # We create a list of all the integers that we want to check,
    # and divide them in an efficient way.
    batches = [[] for ii in range(m)]
    for ii in range(2,n + 1):
        batches[ii % m].append(ii)
    
    # We create a list to hold the process objects
    processes = []
    
    # We start the processes
    for ii in range(m):
        process = mp.Process(target = count_primes, args = (shared_integer, 
                                                           batches[ii]))
        processes.append(process)
        process.start()
    
    # We wait for all processes to finish
    for process in processes:
        process.join()
    
    print(f"There are {shared_integer.value} primes between 2 and {n}")

In [ ]:
n = int(input("Until which number? > ")) #76549, 76429
m = int(input("How many processors? > ")) #78498,76426,76482,74624
%timeit parallel_count_primes(n, m)

76473, 76460, 77957, 1.76 s ± 15.5 ms

Are we getting different values for the same input? Why?

`Value` values are locked by default. This is correct in the sense that even if an assignment consists of multiple operations (such as assigning a string which might be many characters) then this assignment is atomic. However, when incrementing a counter you'll still need an external lock as provided in my example, because incrementing loads the current value and then increments it, and then assigns the result back to the Value.

So without an external lock, you might run into the following circumstance.

- Process 1 reads (atomically) the current value of the counter, then increments it before Process 1 can assign the incremented counter back to the Value, a context switch occurs
- Process 2 reads (atomically) the current (unincremented) value of the counter, increments it, and assigns the incremented result (atomically) back to Value
- Process 1 assigns its incremented value (atomically), blowing away the increment performed by Process 2.

Therefore, let's explicitly lock the integer variable.

In [ ]:
counter_lock = mp.Lock()

def count_primes(shared_integer:mp.Value, numbers_list:list):
    """
    This function counts how many integers are in numbers_list, and add them
    to the shared variable shared_integer.
    """
    for number in numbers_list:
        isPrime = True # We assume that is prime
        for factor in range(2, int(number**0.5)+1):
            if number % factor == 0:
                isPrime = False
                break
        if isPrime:
            with counter_lock:
                shared_integer.value += 1

In [ ]:
n = int(input("Until which number? > "))
m = int(input("How many processors? > "))
%timeit parallel_count_primes(n, m)

This looks good.

In general, `mp.Value` only works for datatypes with fixed memory size. If we require to use a dynamic datatype, we need to use the `mp.Array` object. For example, to define a shared list, we can do:

```python
lista = multiprocessing.Array('b', [True] * (n + 1))
```
Try to use this idea to get all the prime numbers between $2$ and $n$ using parallel programming.

In [ ]:
def count_primes(shared_integer:mp.Value, numbers_list:list):
    """
    This function counts how many integers are in numbers_list, and add them
    to the shared variable shared_integer.
    """
    for number in numbers_list:
        isPrime = True # We assume that is prime
        for factor in range(2, int(number**0.5)+1):
            if number % factor == 0:
                isPrime = False
                break
        if isPrime:
            shared_integer.value += 1
def parallel_count_primes(n:int, m:int):
    """
    This function uses parallel programming (shared memory architecture)
    to count how many primes are between 0 and n, using m processors.
    """
    # We define first the shared integer
    shared_integer = [mp.Value("i", 0) for ii in range(m)] # (i)nteger, starting at 0
    
    # We create a list of all the integers that we want to check,
    # and divide them in an efficient way.
    batches = [[] for ii in range(m)]
    for ii in range(2,n + 1):
        batches[ii % m].append(ii)
    
    # We create a list to hold the process objects
    processes = []
    
    # We start the processes
    for ii in range(m):
        process = mp.Process(target = count_primes, args = (shared_integer[ii], 
                                                           batches[ii]))
        processes.append(process)
        process.start()
    
    # We wait for all processes to finish
    for process in processes:
        process.join()
    
    return shared_integer

In [ ]:
%timeit lista = parallel_count_primes(1000000,8)

In [ ]:
%timeit sum([ii.value for ii in lista])

## Distributed Memory
The easiest kind of distributed memory architecture is the one that does not require communication between the processes. See the following case for example:


In [ ]:
import concurrent.futures

def square_number(number):
    return number * number

if __name__ == "__main__":
    numbers = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    # Create a thread pool with 4 threads
    with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
        # Submit the tasks for squaring numbers
        results = [executor.submit(square_number, num) for num in numbers]

        # Retrieve the results as they are completed
        squared_results = [result.result() for result in concurrent.futures.as_completed(results)]

    print("Squared numbers:", squared_results)

Try to do a similar program where you return how many times you had to apply the [Collatz's function](https://en.wikipedia.org/wiki/Collatz_conjecture) to the first $n$ integers to get 1.

On the other hand, if we require to pass messages between the processes, we should use other library, named `MPI` (message passing interface). See the file `mpi.py` in this folder. You can execute it using
```bash
mpiexec -n 2 python mpi.py
```
That is, we will use 2 processes to run the Python file `mpi.py`.

In [ ]:
from os import system
system("less mpi.py")

In [ ]:
system("mpiexec -n 2 python mpi.py")

Nevertheless, this is a bit boring. Let's see a more interesting example. See `monte_carlo_pi.py`.

In [ ]:
system("cat monte_carlo_pi.py")

In [ ]:
system("mpiexec -n 10 python monte_carlo_pi.py")

Now try to do a program of this kind that estimates the Euler's constant without using explicitly the exponential or logarithmic functions. You should decide which algorithm to use.